In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os

features = ['Okapi', 'TF', 'NormTF', 'BERT', 'LM', 'ENT', 'FracStop', 'StopCover', 'LEN']
labels = ['L%3C=W', 'L%3EW']
user_winner = ['user', 'winner']

NUM_PERMUTATIONS = int(1e5)
def permutation_test(arr1, arr2, num_permutations=NUM_PERMUTATIONS):
    # print(len(arr1), len(arr2))
    observed_diff = np.abs(arr1.mean() - arr2.mean())  # Calculate the difference between the means of the two lists
    combined = np.concatenate([arr1, arr2])
    arr1_shape = arr1.shape[0]
    count = 0
    # for _ in tqdm(range(num_permutations)):
    for _ in range(num_permutations):
        np.random.shuffle(combined)
        permuted_list1 = combined[:arr1_shape]
        permuted_list2 = combined[arr1_shape:]
        assert len(permuted_list1) == len(arr1) and len(permuted_list2) == len(arr2), 'Permutation error'
        assert len(permuted_list1) > 0 and len(permuted_list2) > 0, 'Permutation error'
        permuted_diff = np.abs((np.mean(permuted_list1) - np.mean(permuted_list2)))
        if permuted_diff >= observed_diff:
            count += 1
    p_value = count / num_permutations
    return p_value

In [6]:
df = pd.read_csv(r"feature_matrix.csv")
df['query_id'] = df['query_id'].apply(lambda x: str(x).zfill(3))
QUERIES_A = ['034', '048', '069', '167', '180', '182', '193', '195', '009', '017', '029', '045', '059', '078', '098']
bd = [0,1]
for feature in features:
    b_all = []
    d_all = []
    print(feature)
#     fig = plt.figure()
    graph_b = []
    graph_d = []
    for t in bd:
        df2 = df.copy()
        df2 = df2[(df2['type'] == t) & (df2['query_id'].isin(QUERIES_A))]
        for round_num in range(2, df2["round"].max() + 1):
            q = []
            for query_id in df2['query_id'].unique():
                rounds_df = df2[(df2["query_id"] == query_id) & ((df2["round"] == round_num) | (df2["round"] == round_num - 1)) & (df2["position"] == 1)]
                groups = rounds_df.groupby('query_id')
                if len(groups['user'].unique().values[0]) != 1:
                    x = rounds_df[rounds_df['round'] == round_num-1][feature].values[0]
                    y = rounds_df[rounds_df['round'] == round_num][feature].values[0]
                    q.append(np.abs(x-y))
                    if(t==0):
                        b_all.append(np.abs(x-y))
                    if(t==1):
                        d_all.append(np.abs(x-y))
            if(t == 0):
                graph_b.append(np.mean(q))
            if(t == 1):
                graph_d.append(np.mean(q))
    print('b', len(b_all), 'd', len(d_all))
    print('b', round(np.mean(b_all), 2), 'd', round(np.mean(d_all), 2), permutation_test(np.array(b_all), np.array(d_all)))
    plt.figure(figsize=(6, 6.5))
    plt.plot(range(1,7), graph_b, label='Relevance', color='blue')
    plt.plot(range(1,7), graph_d, label='Diversity', color='orange')
    plt.xlabel('Round', fontsize=14)
    splt.ylabel('Absolute difference', fontsize=14)
#     plt.title(feature)
    plt.legend(fontsize=14)
    plt.savefig(r'cons_win\distance_{0}.png'.format(feature), format='png')
    plt.show()